# BHT Data Applications project
# Automatic Anime recommendation Algorithm
### This project aims to create an algorithm that can determine what anime to recommend to a user.
##### Authors: Rashmi Di Michino and Antonin Mathubert

The 320000 users and 16000 animes dataset was taken from https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020 <br>
We are going to use this dataset to build a model that can recommend an anime based on the animes that the user is watching, has dropped, has kept on hold or put on their watching list.

### 1. Importing and parsing the data
First, we want to import all of our available data in a suitable manner so it is treatable for the next steps of the project.<br><br>
Here, we are going to propose two methods to do that. The first one is very slow but the "clean" way to read json data. Because we don't actually need all the data from the files, we are going to use a second hand made method that is significantly faster.

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import random
import json
import time
import os
import re

In [3]:
dataset_chunks = pd.read_csv("dataset/anime/animelist.csv", chunksize=10000)
chunks = []
for chunk in dataset_chunks:
    chunks.append(chunk)

dataset = pd.concat(chunks[:10], ignore_index=True)
    

In [ ]:
dataset = pd.read_csv("C:/Users/rashm/OneDrive/Desktop/data_applications_project/julius/anime_dataset/animelist.csv")

In [4]:
dataset.drop(['watching_status', 'rating', 'watched_episodes'], axis=1, inplace=True)
dataset = dataset[dataset['user_id'] != 61960]

In [5]:
dataset.head(100)

,user_id,anime_id
0,0,67
1,0,6702
2,0,242
3,0,4898
4,0,21
...,...,...
95,1,3972
96,1,481
97,1,22199
98,1,6547


In [6]:
dataset = dataset.pivot(index='user_id', columns='anime_id', values='anime_id')

In [8]:
dataset[dataset.notnull()] = True

In [10]:
dataset = dataset.fillna(False)

In [11]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets  = apriori(dataset, use_colnames=True)

frequent_itemsets 

,support,itemsets
0,0.593548,(20)
1,0.512903,(199)
2,0.764516,(1535)
3,0.590323,(1575)
4,0.506452,(2167)
5,0.603226,(4224)
6,0.677419,(5114)
7,0.587097,(6547)
8,0.512903,(6746)
9,0.603226,(9253)


In [13]:
frequent_itemsets = apriori(dataset, min_support=0.2, use_colnames=True)

frequent_itemsets 

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets)

rules